## Code developed by Varun Bopardikar

# This is the notebook you can use to get the csv file containing all service requests between 2017 and 2019, along with their ElapsedTime and ElapsedDays.

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import date

In [2]:
#Adds ElaspedTime and ElapsedDays column. 


def convertTime(string):
    """Converts a string to a datetime object.
    """
    time = datetime.strptime(string,'%m/%d/%Y %I:%M:%S %p') 
    return time

def convertDays(string):
    """
    Converts string to date object. 
    """
    time = datetime.strptime(string[:10],'%m/%d/%Y') #Accepts time string, converts it to datetime object.
    return time

def convertFromSeconds(s): # total seconds
    """ convertFromSeconds(s): Converts an integer # of seconds into a list of [days, hours, minutes, seconds]
        input s: an int
    """
    s = s*60
    days = s // (24*60*60)  # total days
    s = s % (24*60*60) # remainder s
    hours = s // (60*60) # total hours
    s = s % (60*60) # remainder s
    minutes = s // 60 # total minutes
    s = s % 60 # remainder s
    statement = (days, ' days') + (hours, ' hrs') +(minutes, ' mins') + (s, 'sec')
    return statement
    
def elapsedTime(csv2017, csv2018, csv2019):
    """
    Accepts CSV files containing 2017-2019 service requests and creates new dataframe of all timestamps for both service request submission and fulfillment times.
    Input CSV file in the format: r'filepath'
    Data source: https://data.lacity.org/A-Well-Run-City/MyLA311-Service-Request-Data-2019/pvft-t768
    """
    df = pd.concat(map(pd.read_csv, [csv2017, csv2018, csv2019]), ignore_index = True)
    
    hdf = df.dropna(axis=0, subset=['CreatedDate', 'ClosedDate'])

    #ElapsedTime 
    df1 = hdf['ClosedDate'].apply(convertTime, 0)
    df2 = hdf['CreatedDate'].apply(convertTime, 0)
    
    hdf['ElapsedTime'] = df1 - df2  
    hdf['ElapsedTime'] = hdf['ElapsedTime']/np.timedelta64(1,'m') 
    hdf['ElapsedTime'] = hdf['ElapsedTime'].apply(convertFromSeconds, 0)   
    
    #ElapsedDays
    df3 = hdf['CreatedDate'].apply(convertDays, 0)
    df4 = hdf['ClosedDate'].apply(convertDays, 0) 
    hdf['ElapsedDays'] = (df4 - df3).dt.days
    
    #Column for Closed Dates
    hdf['Just Date'] = df3
    
    return hdf.reset_index(drop = True)


In [3]:
edf = elapsedTime('service2017.csv', 'service2018.csv', 'service2019.csv')

/Users/varunbopardikar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Users/varunbopardikar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/varunbopardikar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/varunbopardikar/anaconda3/lib/python3.7/site-packag

In [8]:
edf = edf[edf['ElapsedDays'] < 1100]   #Some of the service requests had dates which were mislabelled, resulting in impossible ElapsedDay values. This command gets rid of those requests. 

In [25]:
edf.to_csv('fservice.csv', index = False)

In [26]:
df = pd.read_csv('fservice.csv')

/Users/varunbopardikar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
df.head()

,APC,ActionTaken,Address,AddressVerified,Anonymous,ApproximateAddress,AssignTo,CD,CDMember,ClosedDate,...,StreetName,Suffix,TBMColumn,TBMPage,TBMRow,UpdatedDate,ZipCode,ElapsedTime,ElapsedDays,Just Date
0,West Los Angeles APC,SR Created,"222 N ASHDALE PL, 90049",Y,Y,N,NaN,5.0,Paul Koretz,01/01/2017 09:26:00 AM,...,ASHDALE,PL,H,631.0,1.0,01/01/2017 09:26:00 AM,90049,"(0.0, ' days', 9.0, ' hrs', 25.0, ' mins', 0.0...",0,2017-01-01
1,East Los Angeles APC,SR Created,"1624 N ALLESANDRO ST, 90026",Y,Y,N,CCAC,13.0,Mitch O'Farrell,01/12/2017 05:10:00 PM,...,ALLESANDRO,ST,E,594.0,6.0,01/12/2017 05:10:00 PM,90026,"(11.0, ' days', 16.0, ' hrs', 25.0, ' mins', 0...",11,2017-01-01
2,West Los Angeles APC,SR Created,"250 N ASHDALE AVE, 90049",Y,Y,N,NaN,5.0,Paul Koretz,01/01/2017 09:27:00 AM,...,ASHDALE,AVE,H,631.0,1.0,01/01/2017 09:27:00 AM,90049,"(0.0, ' days', 8.0, ' hrs', 41.0, ' mins', 0.0...",0,2017-01-01
3,North Valley APC,SR Created,"21824 W DEVONSHIRE ST, 91311",Y,N,N,WVA,12.0,Mitchell Englander,01/04/2017 05:33:00 PM,...,DEVONSHIRE,ST,A,500.0,4.0,01/04/2017 05:33:00 PM,91311,"(3.0, ' days', 16.0, ' hrs', 32.0, ' mins', 0....",3,2017-01-01
4,North Valley APC,SR Created,"21230 W DEVONSHIRE ST, 91311",Y,N,N,WVA,12.0,Mitchell Englander,01/04/2017 05:34:00 PM,...,DEVONSHIRE,ST,B,500.0,4.0,01/04/2017 05:34:00 PM,91311,"(3.0, ' days', 16.0, ' hrs', 25.0, ' mins', 0....",3,2017-01-01
